In [11]:
"""
Sequence Number : 4
Ravina Lad
Date : 11th March 2022
CS 5330 Computer Vision
Spring 2021

This Python file includes

- Task 3 : Create a digit embedding space
- Task A : Create a greek symbol data set
- Task B : Create a truncated model
- Task C : Project the greek symbols into the embedding space
- Task D : Compute distances in the embedding space
- Task E : Create your own greek symbol data

- Task Extension : Try more greek letters and build an actual KNN classifier that can take in any square image and classify it.
"""

import sys
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import cv2
import numpy as np
import torchvision.transforms as transforms
from math import sqrt
from sklearn.neighbors import KNeighborsClassifier
from torch.utils.data import DataLoader, Dataset

#Netwrok Defination
class MyNetwork(nn.Module):
    def __init__(self):
        super(MyNetwork, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

#Netwrok Defination
class GreekNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        return x

class Greek_Submodel(MyNetwork):

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    # override the forward method
    def forward( self, x ):
        x = F.relu( F.max_pool2d( self.conv1(x), 2 ) ) # relu on max pooled results of conv1
        x = F.relu( F.max_pool2d( self.conv2_drop( self.conv2(x)), 2 ) ) # relu on max pooled results of dropout of conv2
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        return x

# calculate the Euclidean distance between two vectors
def euclidean_distance(row1, row2):
	distance = 0.0
	for i in range(len(row1)-1):
		distance += (row1[i] - row2[i])**2
	return sqrt(distance)

# main function to run process of loading MNIST data set TASK 1 F and G
def main(argv):

    #declaration of all Hyperparameters used
    batch_size_test = 1000

    #repeatable newtwork
    random_seed = 42
    torch.backends.cudnn.enabled = False
    torch.manual_seed(random_seed)

    greek_loader = torch.utils.data.DataLoader(
                  torchvision.datasets.ImageFolder('/home/ravina/Desktop/OpenCV/MNIST/custom_greek/',
                  transform = transforms.Compose([
                              transforms.Resize([28,28]),
                              transforms.Grayscale(num_output_channels=1),
                              transforms.ToTensor(),
                              transforms.RandomInvert(1),
                              torchvision.transforms.Normalize((0.1307,), (0.3081,)),
                              
    ]) ),
    batch_size=batch_size_test, shuffle=True)
    greek_examples = enumerate(greek_loader)
    batch_idx, (greek_example_data, greek_example_targets) = next(greek_examples) #get next test data
    
    print("Greek = ",greek_examples)


    greek_network = Greek_Submodel()
    greek_network.eval()
    print(greek_network)

    # load the internal state of the network and optimizer when we last saved them.
    network_state_dict = torch.load('/home/ravina/Desktop/OpenCV/MNIST/modelFiles/model.pth')
    greek_network.load_state_dict(network_state_dict)


    with torch.no_grad():
      greek_output = greek_network(greek_example_data)

    print(greek_output.shape)

    greek_example_targets = np.reshape(greek_example_targets,(27,1))

    ssd = 0
    #here dependent oon the alpha, beta , gamma I changed the index value of greek_out and saved the chnages
    ssd = torch.cat((np.square(greek_output[:,None] - greek_output[6]).sum(axis=2), greek_example_targets), 1)
    print(ssd[ssd[:, 0].sort()[1]])

    #To get the KNN score
    neigh = KNeighborsClassifier(n_neighbors=3)
    neigh.fit(greek_output, greek_example_targets)
    print("score = ",neigh.score(greek_output, greek_example_targets))
    return

if __name__ == "__main__":
    main(sys.argv)

Greek =  <enumerate object at 0x7fea944f2e40>
Greek_Submodel(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)
torch.Size([27, 50])
tensor([[0.0000, 0.0000],
        [0.0169, 0.0000],
        [0.0521, 2.0000],
        [0.0542, 0.0000],
        [0.1141, 0.0000],
        [0.1267, 0.0000],
        [0.1428, 2.0000],
        [0.1842, 2.0000],
        [0.2555, 2.0000],
        [0.2717, 2.0000],
        [0.3161, 0.0000],
        [0.3679, 2.0000],
        [0.4121, 2.0000],
        [0.4479, 2.0000],
        [0.5002, 0.0000],
        [0.5799, 0.0000],
        [0.5947, 0.0000],
        [0.6466, 1.0000],
        [0.7182, 1.0000],
        [0.7664, 1.0000],
        [0.7672, 1.0000],
        [0.8312, 1.0000],
        [0.9758, 2.0000],
        [1.0084, 1.0000],
 

/home/ravina/.local/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
